# Analysis of XPN's AtoZ Playlist

One of the cool things about [WXPN](http://xpn.org)
is their frequent special programming which is always interesting.
This year, they put together a multi-day alphabetical playlist
called [XPN's A to Z](http://xpn.org/music-artist/xpn-a-z).
As has been the case for a lot of these events,
the [#XPNAtoZ](https://twitter.com/search?q=%23XPNAtoZ) tweet stream
is as interesting as the music.
Initially I was confused by why
Johnny Cash's _A Boy Named Sue_ was the 3rd song,
when I'd have filed it in the Bs.
So, after getting involved in a 
[twitter exchange](https://twitter.com/brianjgainor/status/803966913483329536)
on the topic of treating articles as significant,
an idea came to me.
I could look at the data to see how words that typically live on stop lists
skewed the distribution of the music.
Along the way other interesting explorations might turn up.
What follows is the result.

In [ ]:
%matplotlib inline
from IPython.display import display, HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide();
    } else {
        $('div.input').show();
    }
        code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


## Status

- At this point, after 18 days we're at Z.
  No more surprise.
  Well unless we dig them out of the data.
- So far the obvious aggregations have been done.
  But there still may be interesting opportunities to find patterns. 
- Graphs have gotten a lot less ugly
  since moving to [Seaborn](http://seaborn.pydata.org/) for plotting
  and spending a bit of time tidying things up.
- I've figured out my issues with [MusicBrainz](https://musicbrainz.org/)
  integration, so counts by year are back.  And not clearly wrong
  this time.

## Load the data

The [AtoZ Playlist page](http://xpn.org/music-artist/xpn-a-z)
contains a directory of songs by first letter.
Behind the scenes, it makes ReST request to their backend.
Originally I said I ought to cache the data.
Mostly I was concerned about it going away.
But as I integrate data from 
[MusicBrainz](https://musicbrainz.org/),
which rate-limits requests, caching becomes important,
though it adds complexity.
Data can be pre-cached by running `load_xpn_cache.py`

In [ ]:

from lxml import html
import requests
import pandas as pd
import os.path as path
import csv

cache_dir = './cache'
xpn_cache = path.join(cache_dir, 'xpn')

alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
letters_so_far = list(alphabet)[:26]

playlist = pd.DataFrame(None, columns = ('Title', 'Artist'))
for letter in letters_so_far:
    cache_file = path.join(xpn_cache, '%s.csv' % letter)
    if path.exists(cache_file):
        df = pd.read_csv(cache_file)
    else:
        rows = []
        page = requests.get('http://xpn.org/static/az.php?q=%s' %  letter)
        tree = html.fromstring(page.content)
        songs = tree.xpath('//li/text()')
        for song in songs:
            rows.append(song.split(' - ', 1))
        df = pd.DataFrame(rows, columns = ('Title', 'Artist'))
        df.to_csv(cache_file, index=False)
    playlist = playlist.append(df, ignore_index=True)


### Data Cleanup and Enrichment

First of all there are some duplicates in the data set.
We'll clean those up before doing anything else.

In [ ]:
HTML(playlist.loc[playlist.duplicated()].to_html())

So we delete those.  
We also add a few artificial columns to make reporting easier.
First Letter and First Word are, well the first letter and word of the title.

For what it's worth, "first word" turns out to be a harder problem that it might seem.
For way more detail than a normal human would want,
see the [Word Splitting](Word Splitting.ipynb) notebook.
Short version, any sequence of letters or apostrophes is a word
for our purposes.

We can also do sentiment analysis on the titles.
Song titles are not exactly idiomatic English,
ideally we would build a training model manually.
But for expediency we can try 
[TextBlob](https://textblob.readthedocs.io/en/dev/)
a convenience library that sits on top of
[NLTK](http://www.nltk.org/), and comes with a pre-trained model.

Finally we can search [MusicBrainz](https://musicbrainz.org/)
for likely recordings matching the artist and title
and use that to make a good guess 
at the year the song was first published.
Because the MusicBrainz interface is rate limited,
this can be slow.
Running the whole play list is over an hour.
So we're only use pre-cached data here.
To see how it gets there see `load_mb_cache.py`.

In [ ]:
# Drop dups
playlist = playlist.drop_duplicates()

# First is the first letter of the title
playlist = playlist.join(playlist.apply(lambda x: x[0][0].upper(), axis=1).to_frame('First Letter'))

from nltk.tokenize import RegexpTokenizer
custom_tokenize = RegexpTokenizer("[\w'\-]+|[^\w'\s\-]").tokenize

# First Word, extract the 'first word' of the title
playlist = playlist.join(playlist.apply(lambda x: custom_tokenize(x[0])[0], axis=1).to_frame('First Word'))

# Sentiment Analysis
# Polarity runs -1.0 (negative) ... +1.0 (postive) sentiment
# Subjectivity runs 0.0 .. 1.0
# Note apply needs to run on a 2xN matrix to treat the result
# as a dataframe, otherwise there's no point in selecting the 
# Artist column
from textblob import TextBlob
sentiment = playlist[['Title', 'Artist']].apply(lambda x: list(TextBlob(x[0]).sentiment),
                           axis=1)
sentiment.columns = ['Polarity', 'Subjectivity']
playlist = playlist.join(sentiment)

# Add Album data from MusicBrainz
# This api is rate limited to about 1 request/sec
# running the whole list will take well over an hour
# we rely completely on pre-loading the cache here
# see load_mb_cache.py for how stuff gets there
mb_cache = path.join(cache_dir, 'musicbrainz', 'song_years.csv')
if path.exists(mb_cache):
    years = pd.read_csv(mb_cache)
else:
    years = pd.DataFrame(None, columns=('Title','Artist', 'Year'))

# Join that up to a copy of the play list
playlist = playlist.merge(years, how='left')
playlist['Year'] = playlist['Year'].fillna(0.0).astype(int)


So the resulting dataframe has seven fields:
Title, Artist, First Letter, First Word,
Sentiment Polarity and Subjectivity,
and finally Year of publication.
Before diving in, just to get perspective,
here are some basic stats on the data and a dump of the first and last few rows.

In [ ]:
HTML(playlist.describe(include='all', percentiles=[]).to_html(na_rep=''))

In [ ]:
HTML(playlist.head(5).to_html())

In [ ]:
HTML(playlist.tail(5).to_html())

## Data Analysis

### Songs by Letter

It's the simplest thing one can look at.
And I wouldn't think it all that interesting.
But given how uneven the distribution has been,
I thought it might be interesting.
And there have been a few "how long with th I's go"
side conversations.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(6, 6))
sns.set_color_codes('pastel')
sns.countplot(y='First Letter', data=playlist, color='b')
ax.set(xlabel="Songs Played")

### Most Frequent Artists

Even this early in the playlist,
some artists show up often.
This might say something about the music the station plays,
or the listeners as much of this list comes from previous 
playlists like "885 best songs of all time"
or my favorite, the "85 worst songs of all time."
For the moment, let's just look at the most frequent dozen or so.

In [ ]:
c = playlist['Artist'].value_counts()
artists = pd.DataFrame(zip(c.keys().tolist(), c.tolist()),
                       columns=('Artist', 'count'))
f, ax = plt.subplots(figsize=(6, 6))
sns.set_color_codes('pastel')
sns.barplot(y='Artist', x='count', data=artists.head(25), color='b')
ax.set(xlabel="Appearences in the Playlist (so far)")

### Duplicate Titles
From the titles alone, covers and songs that just happen to have the same title
look the same.
But even early on, the number of duplicate titles
was more than I'd have expected.
And a few are more than just duplicated.
In fact, it's crazy that there were four songs called "Crazy",
that that's in a 9 way tie for 3rd place,
and "Home" and "Hold on" have 5 instances each.

What I can't tell is if this all says that there are a fair number of covers in the list,
or if artists just are not that creative at picking names.

#### Frequency Distribution of Titles

Since there are a lot of duplicate titles,
it might be easier to start by looking at the overall distribution.
That is, how many titles  show up 2, 3, or more times.

In [ ]:
# Build a Datafrae of Unique Titles to Appearence counts
c = playlist['Title'].value_counts()
title_counts = pd.DataFrame(zip(c.keys().tolist(), c.tolist()),
                           columns=('Title', 'Count'))
f, ax = plt.subplots(figsize=(6, 3))
sns.set_color_codes('pastel')
sns.countplot(y='Count', data=title_counts[title_counts['Count'] >2],
              color='b')
ax.set(ylabel="Repetitions of a Title", xlabel="Instances in Playlist")

#### Most Popular Titles
Of course we want to know what those titles were.
At least the really popular ones.

In [ ]:
HTML(title_counts[title_counts['Count'] > 3].to_html())

### Most Frequent First Words

This is the question that started me down this path.
How much do common "stop words" like articles or propositions 
skew the distribution?
Or do so many songs start with a small set of words
that no amount of pruning would even the distribution out?
Here's an early look a the top 25 first words.
Yes, "A" *was* up there.
But even early on it wasn't in the lead.
Perhaps my initial concerns were unfounded.
Except that the long reign of "I" as the top first word has ended
as, after almost 18 hours of air time, "The" overtook it,
and and kept on going.
So, in the end the playlist could have been come out different
had articles not be used to index titles.
It's just that "The" mattered a lot more than "A."

In [ ]:
c = playlist['First Word'].value_counts()
words = pd.DataFrame(zip(c.keys().tolist(), c.tolist()),
                       columns=('First Word', 'count'))

f, ax = plt.subplots(figsize=(6, 6))
sns.set_color_codes('pastel')
sns.barplot(y='First Word', x='count', data=words.head(20), color='b')
ax.set(xlabel="Appearences in the Playlist (so far)")

The same data as a word cloud,
though the python `wordcloud` package does a few things work against us
and are not customizable in the released version.
In particular, it kills of the I's and A's.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

firsts = playlist.apply(lambda x: custom_tokenize(x[0])[0], axis=1)
wordcloud = WordCloud(max_font_size=40, stopwords='').generate(' '.join(firsts.tolist()))
plt.figure()
plt.imshow(wordcloud)
plt.axis('off')
plt.show


### Sentiment Analysis
Sentiment Analysis rates text on how positive or negative is.
How happy are the songs XPN plays?
How many are angry (or more likely sad)?
The vast majority of the titles are neutral.

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
sns.distplot(playlist['Polarity'])
ax.set(xlabel="Sentiment", ylabel="Number of Songs")

But it's easier to see the distribution
if we ignore the neutral songs.
Here we see there's a decent spread,
with a reasonably strong skew towards the positive.
Perhaps songwriters often mine pain for themes,
but we prefer to play happy songs.

In [ ]:
f, ax = plt.subplots(figsize=(6, 6))
sns.distplot(playlist[playlist['Polarity'] != 0]['Polarity'])
ax.set(xlabel="Sentiment (omitting zero values)", ylabel="Number of Songs")

What are some of the most positive songs?

In [ ]:
HTML(playlist.sort_values('Polarity', ascending=False)[['Artist', 'Title', 'Polarity']].head(10).to_html())

And the negative?

In [ ]:
HTML(playlist.sort_values('Polarity', ascending=True)[['Artist', 'Title', 'Polarity']].head(10).to_html())

Lots of Evil there.

### When Were the Songs From

Were the songs in the playlist all classic rock
or where they all pretty current?
We can tell by grouping by the Years we've extracted 
via lookups on MusicBrainz.
We can't find every track there,
but we can find 5481 out of 5673 and that's not bad considering.


In [ ]:

c = playlist['Year'].value_counts()
year_counts = pd.DataFrame(zip(c.keys().tolist(), c.tolist()),
                       columns=('Year', 'Count'))
f, ax = plt.subplots(figsize=(6, 14))
sns.set_color_codes('pastel')
sns.barplot(y='Year', x='Count',
            data=year_counts[year_counts['Year'] > 1910],
            orient='h', color='b')
ax.set(xlabel="Number of Songs", ylabel="Year of Publication")


The data for years is based on an imperfect match,
so it might not really be 100%.
But it passes enough of a sniff test that
it's worth using.

There's a strong bias for late 60s and early 70's.
But it's a pretty wide thick distribution.
This is an audience that keeps up with new music.

Oh, those entries for 2017.  They are legitimate, sort of.
_The Castle_ by the Flaming Lips will be on _Oczy Mlody_,
due out Jan 17.
And _Astral Plane_ by Valerie June will be on _The Order of Time_,
due out Jan 27.
Since the methodology we've used is to use the publish date
of the oldest physical release that contained a track for
the same title and the same artist,
we end up treating early downloads as being "from the future."

### How Is A-Z Different

Because it is curated from existing XPN playlists,
the A-Z playlist might not be 100% representative of
the day to day programming.
There is a [http://xpn.org/playlists/xpn-playlist](XPN Playlist) page,
and while it doesn't have a ReST interface for navigation like 
the [http://xpn.org/music-artist/xpn-a-z](A-Z Playlist) page does,
it isn't very hard to screen scrape.
So, we can load say the previous months playlist 
and do some comparisons.

In [ ]:
import requests
import re

history_cache = path.join(cache_dir, 'historical')

historical_playlist = pd.DataFrame(None, columns=('Artist', 'Title'))
days = pd.date_range(start='11/1/2016', end='11/29/2016')
for day in days:
    day_s = "%02d-%02d-%04d" % (day.month, day.day, day.year)
    cache_file = path.join(history_cache, "%s.csv" % day_s)
    if path.exists(cache_file):
        df = pd.read_csv(cache_file)
    else:
        rows = []
        page = requests.post('http://xpn.org/playlists/xpn-playlist',
                             data = {'playlistdate': day_s})
        tree = html.fromstring(page.content)
        tracks = tree.xpath('//h3/a/text()')
        # not all rows are tracks, some are membership callouts
        # but real tracks start with times and are formatted
        # HH:MM [am|pm] Artist - Title
        # Special programs like World Cafe, Echos, ...
        # also start eith time, but don't have useful track info
        # but those list the program inside bars
        # eg |World Cafe| -  "Wednesday 11-2-2016 Hour 2, Part 7"
        date_regex = re.compile("^\d{2}:\d{2}\s")
        for track in tracks:
            if date_regex.match(track) and track[9:10] != '|':
                rows.append(track[9:].split(' - ', 1))
        df = pd.DataFrame(rows, columns = ('Artist', 'Title'))
        df.to_csv(cache_file, index=False)

    historical_playlist = historical_playlist.append(df, ignore_index=True)



#### Which Artists are Over or Under Represented

One thing we can do is take the the artist counts from above
and compare it the historical playlist we just loaded.

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn import preprocessing
c = playlist['Artist'].value_counts()
az_artists = pd.DataFrame(zip(c.keys().tolist(), c.tolist()),
                         columns=('Artist', 'Frequency'))
xform = preprocessing.MinMaxScaler()
c = historical_playlist['Artist'].value_counts()
hist_artists = pd.DataFrame(zip(c.keys().tolist(), c.tolist()),
                           columns=('Artist', 'Frequency'))
artist_freq = az_artists.merge(hist_artists, how='outer',
                               on = 'Artist',
                               suffixes=('_az', '_hist'))
artist_freq[['Frequency_az', 'Frequency_hist']] \
 = xform.fit_transform(artist_freq[['Frequency_az', 'Frequency_hist']].fillna(0))

artist_freq['Difference'] = artist_freq['Frequency_az'] - artist_freq['Frequency_hist']
artist_freq = artist_freq.sort(columns='Difference')


In November the most popular artists were

In [ ]:

f, ax = plt.subplots(figsize=(6, 6))
sns.set_color_codes('pastel')
sns.barplot(y='Artist', x='Frequency', data=hist_artists.head(25), color='b')
ax.set(xlabel="Appearences in November")

Artists played more frequently in the A-Z playlist than the preceding month:

In [ ]:
HTML(artist_freq.head().to_html())

Artists played less frequently in the playlist than the precdeing month:

In [ ]:
HTML(artist_freq.tail().to_html())

#### Which Years Are More Frequent

The A-Z playlist skews heavily to the late 60's and early 70's.
How representative is that of the day to day play list?
A lot of us listen to XPN to be introduced to new music,
yet we fondly remember our high school years.
We can join the historical data from November up against 
the MusicBrainz data as we did above.
This time there are fewer matches and over 800 of 3.5K titles can't be found.
Still it gives us a sense of things.

In [ ]:
mb_hist_cache = path.join(cache_dir, 'musicbrainz', 'hist_years.csv')
if path.exists(mb_hist_cache):
    hist_years = pd.read_csv(mb_hist_cache)
else:
    hist_years = pd.DataFrame(None, columns=('Artist','Title', 'Year'))

# Join that up to a copy of the play list
historical_playlist = historical_playlist.merge(hist_years, how='left')
historical_playlist['Year'] = historical_playlist['Year'].fillna(0.0).astype(int)
HTML(historical_playlist.head().to_html())

We can plot that as we did before.

In [ ]:
c = historical_playlist['Year'].value_counts()
hist_year_counts = pd.DataFrame(zip(c.keys().tolist(), c.tolist()),
                                columns=('Year', 'Count'))
f, ax = plt.subplots(figsize=(6, 14))
sns.set_color_codes('pastel')
sns.barplot(y='Year', x='Count',
            data=hist_year_counts[hist_year_counts['Year'] > 1950],
            orient='h', color='b')
ax.set(xlabel="Number Tracks Played In November", ylabel="Year of Publication")


Well tha's a a lot different!
Goodby 1970, hello 2016.

## Availability

The code for this project is in [my git hub repo](https://github.com/asudell/a2z).
The notebook its self is [published on nbviewer](http://nbviewer.jupyter.org/github/asudell/a2z/blob/master/AtoZ.ipynb)

## License

This project is licensed under a
[Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/).
You are free to use for commercial or non-commercial purposes,
so long as you attribute the source and also allow sharing.

## Related Efforts

Apparently I'm not the only one who thought to do something like this.
Checkout [Bruce Segal's work](https://public.tableau.com/profile/besegal#!/vizhome/BESegalWXPNAtoZPlayListPublished/XPNAtoZFindYourSongfromAtoZ)
as well as [Lena Bartel's work](https://public.tableau.com/profile/lena.bartell#!/vizhome/XPN_AtoZ/XPNA-ZPlaylistData).
They explore the same topics, plus a few more.
And both use Tableau, which is very nice looking.
